# Modelos Ensembles
Katlyn Mackness <br>
17/12/2024 - Updated 25/02/2025

## Objetivo
Efectuar un análisis de clasificación mediante ensambles que permita desarrollar un modelo predictivo basado en distintas métricas aplicadas a una base de datos grande.

## Problema
Imagina que eres un investigador médico que recopila datos para un estudio. Has recopilado datos sobre un conjunto de pacientes, todos ellos con la misma enfermedad. 

Durante su tratamiento, cada paciente ha respondido a uno de los 5 medicamentos:
* drugA
* drugB
* drugC
* drugX
* drugY

Construye un modelo para averiguar qué medicamento podría ser apropiado para un futuro paciente con la misma enfermedad.

In [1]:
# Importar bibliotécas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics, preprocessing
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Cargue la base de datos “drugs.csv”
drugs = pd.read_csv("Data/drugs.csv")
drugs.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,drugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,drugY


In [3]:
feature_cols = ["Age", "Sex", "BP", "Cholesterol", "Na_to_K"]
X = drugs[feature_cols].values
y = drugs.Drug

In [4]:
# Convertir las variables predictoras cualitativas a una escala numérica 
Cod_BP = preprocessing.LabelEncoder()
Cod_BP.fit(['HIGH', 'NORMAL', 'LOW'])
X[:,2] = Cod_BP.transform(X[:,2])

Cod_Cholesterol = preprocessing.LabelEncoder()
Cod_Cholesterol.fit(['HIGH', 'NORMAL'])
X[:,3] = Cod_Cholesterol.transform(X[:,3])

In [5]:
Cod_Sex = preprocessing.LabelEncoder()
Cod_Sex.fit(['F', 'M'])
X[:,1] = Cod_Sex.transform(X[:,1])

In [6]:
X[0:5]

array([[23, 0, 0, 0, 25.355],
       [47, 1, 1, 0, 13.093],
       [47, 1, 1, 0, 10.114],
       [28, 0, 2, 0, 7.798],
       [61, 0, 1, 0, 18.043]], dtype=object)

## Random Forest Classifier

In [7]:
# Crear grupos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)

RandomForestClassifier()

In [8]:
# Score F1 para el grupo de entrenamiento (training)
rf.score(X_train, y_train)

1.0

In [9]:
# Score F1 para el grupo de prueba (testing)
rf.score(X_test, y_test)

0.975

In [10]:
# Estadísticas de desempeño generales
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       drugA       1.00      1.00      1.00         4
       drugB       1.00      1.00      1.00         2
       drugC       1.00      0.75      0.86         4
       drugX       0.93      1.00      0.96        13
       drugY       1.00      1.00      1.00        17

    accuracy                           0.97        40
   macro avg       0.99      0.95      0.96        40
weighted avg       0.98      0.97      0.97        40



## Gradient Boosted Trees

In [11]:
# Crear grupos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# GB con valores predeterminados
gb = GradientBoostingClassifier(n_estimators=100)
gb.fit(X_train, y_train)

GradientBoostingClassifier()

In [12]:
# Score F1 para el grupo de entrenamiento
gb.score(X_train, y_train)

1.0

In [13]:
# Score F1 para el grupo de prueba
gb.score(X_test, y_test)

1.0

In [14]:
# Estadísticas de desempeño general
y_pred = gb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       drugA       1.00      1.00      1.00         4
       drugB       1.00      1.00      1.00         2
       drugC       1.00      1.00      1.00         4
       drugX       1.00      1.00      1.00        13
       drugY       1.00      1.00      1.00        17

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40



In [16]:
# El modelo parece sobreajustarse
# Encontrar hiperparametros optimos
parameters = {
    'n_estimators': [50, 100, 200],
    'max_depth': [2, 3, 4],
    'learning_rate': [0.01, 0.1, 0.2]
}

gb = GradientBoostingClassifier(random_state=1)

cv = GridSearchCV(gb, parameters, cv=5)
cv.fit(X_train, y_train)

best_params = cv.best_params_
print("Best parameters found: ", best_params)

Best parameters found:  {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 50}


In [19]:
gb_best = GradientBoostingClassifier(n_estimators=100,
                                     max_depth=2,
                                     learning_rate=0.01,
                                     random_state=1)
# Entrenar
gb_best.fit(X_train, y_train)

# Evaluar
y_pred = gb_best.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       drugA       1.00      1.00      1.00         4
       drugB       1.00      1.00      1.00         2
       drugC       1.00      0.75      0.86         4
       drugX       0.93      1.00      0.96        13
       drugY       1.00      1.00      1.00        17

    accuracy                           0.97        40
   macro avg       0.99      0.95      0.96        40
weighted avg       0.98      0.97      0.97        40



## AdaBoost Classifier

In [20]:
# Crear grupos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [21]:
# AdaBoost usa arboles de decision como clasificador por default
abc = AdaBoostClassifier(n_estimators=50, learning_rate=1)

# El entrenamiento del clasificador AdaBoost
model = abc.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [22]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       drugA       0.67      1.00      0.80         4
       drugB       0.00      0.00      0.00         2
       drugC       0.00      0.00      0.00         4
       drugX       0.76      1.00      0.87        13
       drugY       1.00      1.00      1.00        17

    accuracy                           0.85        40
   macro avg       0.49      0.60      0.53        40
weighted avg       0.74      0.85      0.79        40



## Conclusiones
Según las métricas:

* Gradient Boosting Classifier tiene la mayor precisión (1,00) y precisión, recuperación y puntuación f1 perfectas para todas las clases.
* Random Forest Classifier también tiene un muy buen rendimiento con una precisión de 0,97 y alta precisión, recuperación y puntuaciones f1, pero ligeramente inferior a Gradient Boosting.
* AdaBoost Classifier tiene el rendimiento más bajo con una precisión de 0,85 y precisión, recuperación y puntuaciones f1 inferiores para algunas clases.

### Mejor modelo
El clasificador *Gradient Boosting* es el mejor modelo en este caso, ya que logra puntuaciones perfectas en todas las métricas.